In [1]:
import pandas as pd
import datetime
df = pd.read_csv( "castle-event-data-v4.csv",sep=',', header=0, parse_dates=True,quoting=1 )


C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import numpy as np

In [3]:
#replace all values "custome" in column name with custom name
df.name=df.name.replace(to_replace='custom',value=np.nan)
df.name=df.name.fillna(df.custom_name)

In [75]:
df_agent_os=pd.read_csv('agent_name.csv')

In [76]:
df_agent_os.columns.values

array(['Unnamed: 0', '0', '1'], dtype=object)

In [77]:
df_agent_os.rename(columns={'Unnamed: 0':'ua','0':'os','1':'agent_name'},inplace=True)

In [78]:
df_agent_os.head()

,ua,os,agent_name
0,Mozilla/5.0 (Linux; Android 5.1.1; Ilium_PAD_i...,Android,Android Webkit Browser
1,Mozilla/5.0 (Linux; Android 4.4.2; SM-G355M Bu...,Android,Android Webkit Browser
2,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,Windows NT,Internet Explorer
3,Mozilla/5.0 (Linux; Android 5.0.1; SM-N910P Bu...,Android,Android Webkit Browser
4,Mozilla/5.0 (Linux; Android 4.4.4; ILIUM L900 ...,Android,Android Webkit Browser


In [85]:
df=df.merge(df_agent_os,left_on='ua',right_on='ua',how='left')

In [87]:
df_model=df.drop(['ua','created_at','user_id','session_id','device_id','ip','lon','lat','accuracy','custom_name'],axis=1)
df_model=df_model.drop(['l','city','isp_name'],axis=1)

In [88]:
df_model.columns

Index([u'name', u'continent', u'country', u'datacenter', u'proxy',
       u'mobile_device', u'score', u'os_x', u'agent_name_x', u'os_y',
       u'agent_name_y'],
      dtype='object')

In [89]:
df.name.value_counts()

page_viewed                                   1535114
login_succeeded                                335810
passwordChange_succeeded                        73548
Payment With Balance                            62531
Regular Cellphone Confirmation                  26639
passwordReset_requested                         14829
Create Withdraw                                 13383
Account Create Edc Deposit On Landing Page      12417
registration_succeeded                          11175
Card Checkout                                    7152
Mobile Show Cvv                                  4975
Cellphone Confirmation                           3864
First Timeline Access                            3580
First Card Access                                3576
Mobile Withdraw Check                            1728
New Contact                                       463
Account Login Soft Block                          200
Mobile Account Change Password                      9
Regular Cellphone Confirmati

In [90]:
df_trans=pd.get_dummies(df_model,dummy_na=True)

In [91]:
df_trans.columns

Index([u'datacenter', u'proxy', u'mobile_device', u'score',
       u'name_Account Create Edc Deposit On Landing Page',
       u'name_Account Login Mobile Chat#using Token',
       u'name_Account Login Soft Block', u'name_Card Checkout',
       u'name_Cellphone Confirmation', u'name_Create Withdraw',
       ...
       u'agent_name_y_Opera', u'agent_name_y_Opera Mini', u'agent_name_y_PHP',
       u'agent_name_y_Palemoon', u'agent_name_y_RockMelt',
       u'agent_name_y_Safari', u'agent_name_y_SeaMonkey',
       u'agent_name_y_UC Browser', u'agent_name_y_unknown',
       u'agent_name_y_nan'],
      dtype='object', length=218)

In [92]:
df_trans.shape

(2111004, 218)

In [93]:
# apparently we need to do 1 hot coding for the model
df_trans=pd.get_dummies(df_model,dummy_na=True)

In [94]:
df_trans.head()

,datacenter,proxy,mobile_device,score,name_Account Create Edc Deposit On Landing Page,name_Account Login Mobile Chat#using Token,name_Account Login Soft Block,name_Card Checkout,name_Cellphone Confirmation,name_Create Withdraw,...,agent_name_y_Opera,agent_name_y_Opera Mini,agent_name_y_PHP,agent_name_y_Palemoon,agent_name_y_RockMelt,agent_name_y_Safari,agent_name_y_SeaMonkey,agent_name_y_UC Browser,agent_name_y_unknown,agent_name_y_nan
0,False,False,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,False,False,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,False,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,False,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,False,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#initialize isolation forest
from sklearn.ensemble import IsolationForest
seed=25
sf=IsolationForest(contamination=0.001,n_jobs=-1,random_state=seed)

In [ ]:
# fitting the isolation forest model
sf.fit(df_trans)


In [ ]:
predict_anom=sf.predict(df_trans)

In [ ]:
df_trans['score']=predict_anom

In [ ]:
df['score']=predict_anom

In [ ]:
df_trans.head()

In [ ]:
df_anom=df[df.score==-1]

In [ ]:
# columns in model: 'name', u'continent', u'country', u'datacenter', u'proxy',
#       u'mobile_device'
df_anom.name.value_counts()

In [ ]:
df_anom.mobile_device.value_counts()

In [ ]:
df.name.value_counts()

In [ ]:
df_anom.country.value_counts()

In [ ]:
df.shape